In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [22]:
addplant = pd.read_csv("add-plant.csv")
addplant.head()

,participant_id,condition,reaction_time,major_errors,minor_erros,clicks,confidence,outcome_certainty,perceived_speed,navigation_clarity,error_recovery_ease,group
0,2,A,6952,0,0,6,7,4,7,7,7,0
1,2,B,1862,0,0,3,7,7,7,7,7,0
2,3,A,4198,2,0,5,4,7,6,7,4,0
3,3,B,6498,0,0,3,6,7,6,6,7,0
4,12,A,4004,1,0,3,7,7,7,7,6,0


In [23]:
addplant["condition"] = addplant["condition"].astype("category")
addplant.dtypes

participant_id            int64
condition              category
reaction_time             int64
major_errors              int64
minor_erros               int64
clicks                    int64
confidence                int64
outcome_certainty         int64
perceived_speed           int64
navigation_clarity        int64
error_recovery_ease       int64
group                     int64
dtype: object

## Paired t-test for A vs B

reaction time


In [20]:
wide = addplant.pivot(index="participant_id", columns="condition", values="reaction_time")
t, p = stats.ttest_rel(wide["A"], wide["B"])
print("paired t-test reaction_time: t =", t, "p =", p)
print("means: A =", wide["A"].mean(), "B =", wide["B"].mean())

paired t-test reaction_time: t = 1.310539641559048 p = 0.21671645266788628
means: A = 5751.416666666667 B = 4393.916666666667


## order effect check

reaction time

In [25]:

wide = addplant.pivot_table(
    index="participant_id",
    columns="condition",
    values="reaction_time",
    aggfunc="mean"
).reset_index()

groups = addplant[["participant_id", "group"]].drop_duplicates("participant_id")
wide = wide.merge(groups, on="participant_id", how="left")

wide = wide.dropna(subset=["A", "B"])

wide["diff_A_minus_B"] = wide["A"] - wide["B"]

g0 = wide.loc[wide["group"] == 0, "diff_A_minus_B"]
g1 = wide.loc[wide["group"] == 1, "diff_A_minus_B"]

print("n group0 =", len(g0), "n group1 =", len(g1))
print("mean diff group0 =", g0.mean(), "mean diff group1 =", g1.mean())

t, p = stats.ttest_ind(g0, g1, equal_var=False)  # Welch
print("order effect on diff (Welch): t =", t, "p =", p)

n group0 = 7 n group1 = 5
mean diff group0 = 1904.5714285714287 mean diff group1 = 591.6
order effect on diff (Welch): t = 0.5756785320996402 p = 0.5826856984144262


/var/folders/9f/kctrjggs22ndcx2l59bgyjy00000gn/T/ipykernel_22020/4237587484.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide = addplant.pivot_table(
